<a href="https://colab.research.google.com/github/ensw1031/class-codes/blob/master/webscrapy_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup

import requests

In [ ]:
url = 'https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/'
page = requests.get(url)


<Response [200]>

In [ ]:
page.content

b'<!DOCTYPE html>\n<!--[if lt IE 7 ]><html class="ie ie6" lang="en"> <![endif]-->\n<!--[if IE 7 ]><html class="ie ie7" lang="en"> <![endif]-->\n<!--[if IE 8 ]><html class="ie ie8" lang="en"> <![endif]-->\n<!--[if (gte IE 9)|!(IE)]><!-->\n<html lang="pt-BR" xmlns=\'https://www.w3.org/1999/xhtml\' prefix=\'og: https://ogp.me/ns#\'> \n<!--<![endif]-->\n<head>\n\t<meta charset="iso-8859-1"/>\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"/>\n\t<title>Busca  - Apartamento - Padr&atilde;o - Londrina para loca&ccedil;ao</title>\n    <meta name="description" content="Buscar im&oacute;veis - Apartamento - Padr&atilde;o - Londrina para loca&ccedil;ao" />\n    <meta name="keywords" content="pesquisa de im&oacute;vel, Apartamento , Padr&atilde;o , Londrina, loca&ccedil;ao, venda,im&oacute;veis, imobili&aacute;ria, pesquisar em c&oacute;digo, pesquisa avan&ccedil;ada, tipo do im&oacute;vel, cidade, bairro, pre&ccedil;o" />\n    <meta name="Name" content="Raul

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
soup.prettify()

'<!DOCTYPE html>\n<!--[if lt IE 7 ]><html class="ie ie6" lang="en"> <![endif]-->\n<!--[if IE 7 ]><html class="ie ie7" lang="en"> <![endif]-->\n<!--[if IE 8 ]><html class="ie ie8" lang="en"> <![endif]-->\n<!--[if (gte IE 9)|!(IE)]><!-->\n<html lang="pt-BR" prefix="og: https://ogp.me/ns#" xmlns="https://www.w3.org/1999/xhtml">\n <!--<![endif]-->\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>\n  <title>\n   Busca  - Apartamento - Padrão - Londrina para locaçao\n  </title>\n  <meta content="Buscar imóveis - Apartamento - Padrão - Londrina para locaçao" name="description"/>\n  <meta content="pesquisa de imóvel, Apartamento , Padrão , Londrina, locaçao, venda,imóveis, imobiliária, pesquisar em código, pesquisa avançada, tipo do imóvel, cidade, bairro, preço" name="keywords"/>\n  <meta content="Raul Fulgencio Negócios Imobiliários" name="Name"/>\n  <meta content="Londrina" name="City"/>\n  <meta content="PR" name="

In [ ]:
apts = soup.find_all('div', class_='item col-sm-6 col-md-4 col-lg-3')

In [ ]:
for apt in apts:
  price = apt.find('div', class_='price').find('span').text.strip()
  title = apt.find('div', class_='info').find('h3').find('a').text.strip()
  neighborhood = apt.find('div', class_='info').find('h3').find('small').text.strip()
  print(f'{title} ({neighborhood}) {price}')


Apartamento Padrão (Centro) 1.300,00
Apartamento Padrão (Gleba Fazenda Palhano) 1.850,00
Apartamento Padrão (Gleba Fazenda Palhano) 2.400,00
Apartamento Padrão (Gleba Fazenda Palhano) 2.200,00
Apartamento Padrão (Vila Larsen 1) 750,00
Apartamento Padrão (Gleba Fazenda Palhano) 2.500,00
Apartamento Padrão (Centro) 1.100,00
Apartamento Padrão (Gleba Fazenda Palhano) 1.650,00
Apartamento Padrão (Gleba Fazenda Palhano) 1.650,00
Apartamento Padrão (Centro) 1.600,00
Apartamento Padrão (Champagnat) 1.200,00
Apartamento Padrão (Gleba Fazenda Palhano) 2.200,00
Apartamento Padrão (Centro) 1.050,00
Apartamento Padrão (Gleba Fazenda Palhano) 2.200,00
Apartamento Padrão (Gleba Fazenda Palhano) 1.200,00
Apartamento Padrão (Gleba Fazenda Palhano) 1.350,00
